<a href="https://colab.research.google.com/github/RomainBui/I3M/blob/main/I3M_by_R3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I3M Computation by 3R



This project will perform the mask_RCNN inference of MT in order to eventually compute I3M

In [ ]:
#@title Configuration { run: "auto" }
run_mode = "Inference" #@param ["Inference", "Comparison inference / manual", "I3M on manual segmentation"]
endpoints_detection_method = "Gradient" #@param ["Ray tracing", "Skeleton", "Gradient", "Ray tracing + skeleton", "All"]
HD = "Image already HD" #@param ["Apply HD filter", "Image already HD"]
debug = False #@param {type: "boolean" }

## Select Model + Image

In [ ]:
!pwd

/content


In [ ]:
!rm /content/*.jpg || {}

from google.colab import files

print('please select your Images and/or Manual segmentations')
uploaded = files.upload()
filenames = list(uploaded.keys())
image_filenames = [filename for filename in filenames if filename.endswith('.jpg') or filename.endswith('.png')]
manual_segmentation_filenames = [filename for filename in filenames if filename.endswith('.json')]

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

please select your Images and/or Manual segmentations


Saving 414_l.jpg to 414_l.jpg
Saving 414_l.json to 414_l.json
Saving 417_r.jpg to 417_r.jpg
Saving 417_r.json to 417_r (1).json
Saving 418_r.jpg to 418_r.jpg
Saving 418_r.json to 418_r.json
Saving 419_l.jpg to 419_l.jpg
Saving 419_l.json to 419_l.json
Saving 420_r.jpg to 420_r.jpg
Saving 420_r.json to 420_r.json
Saving 421_r.jpg to 421_r.jpg
Saving 421_r.json to 421_r.json
Saving 427_l.jpg to 427_l.jpg
Saving 427_l.json to 427_l.json
Saving 429_r.jpg to 429_r.jpg
Saving 429_r.json to 429_r.json
Saving 432_l.jpg to 432_l.jpg
Saving 432_l.json to 432_l.json
Saving 433_l.jpg to 433_l.jpg
Saving 433_l.json to 433_l.json
Saving 439_r.jpg to 439_r.jpg
Saving 439_r.json to 439_r.json
Saving 442_l.jpg to 442_l.jpg
Saving 442_l.json to 442_l.json
Saving 444_l.jpg to 444_l.jpg
Saving 444_l.json to 444_l.json
Saving 446_l.jpg to 446_l.jpg
Saving 446_l.json to 446_l.json
Saving 454_r.jpg to 454_r.jpg
Saving 454_r.json to 454_r.json
Saving 470_r.jpg to 470_r.jpg
Saving 470_r.json to 470_r.json
Savi

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [ ]:
downloaded = drive.CreateFile({'id':"10jP0flXeozBQUXcx6Q7CUonaQiTZAjrV"})  # I3M - JUNE 2021
#downloaded = drive.CreateFile({'id':"1CAn1PBg_OrGhN41LP51UNY46zTsi3xo-"})  # mask_rcnn_tooth_segmentation_0006
#downloaded = drive.CreateFile({'id':"19euYX5zo1ty5HacZ0mwXuUoHygFaWLkV"}) # mask_rcnn_tooth_segmentation_0008
#downloaded = drive.CreateFile({'id':"1o6uigkw3ZSN2H1vBRv5-uqLX3UiD7m-w"}) # mask_rcnn_tooth_segmentation_0018
#downloaded = drive.CreateFile({'id':"1eFsvwd7knZBOpUTO_7iwIcipnvE7Cq1W"}) # mask_rcnn_tooth_segmentation_0019

downloaded.GetContentFile('I3M.h5')  

## Install backbone in Google Colab





In [ ]:
#%tensorflow_version 1.x

In [ ]:
!git clone https://github.com/matterport/Mask_RCNN
%cd Mask_RCNN
!pip3 install -r requirements.txt
!python3 setup.py install

fatal: destination path 'Mask_RCNN' already exists and is not an empty directory.
/content/Mask_RCNN
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'license-file' will not be supported in future versions. Please use the underscore name 'license_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'requirements-file' will not be supported in future versions. Please use the underscore name 'requirements_file' instead
  % (opt, underscore_opt))
running install
running bdist_egg
running egg_info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links to mask_rcnn.egg-info/dependency_links.txt
writing top-

In [ ]:
!git clone https://github.com/cocodataset/cocoapi.git
%cd cocoapi/PythonAPI
!make
%cd ../../

fatal: destination path 'cocoapi' already exists and is not an empty directory.
/content/Mask_RCNN/cocoapi/PythonAPI
python setup.py build_ext --inplace
running build_ext
skipping 'pycocotools/_mask.c' Cython extension (up-to-date)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/pycocotools
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-LSlbJj/python3.7-3.7.11=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-LSlbJj/python3.7-3.7.11=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.7/dist-packages/numpy/core/include -I../common -I/usr/include/python3.7m -c ../common/maskApi.c -o build/temp.linux-x86_64-3.7/../common/maskApi.o -Wno-cpp -Wno-unused-function -std=c99
../common/maskApi

In [ ]:
!git clone https://www.github.com/iozzinor/Apex_points
!pip3 install scikit-image==0.18.1

fatal: destination path 'Apex_points' already exists and is not an empty directory.


In [ ]:
!pip3 uninstall -y keras-nightly
!pip3 uninstall -y keras
!pip3 install keras==2.2.5

#!pip3 uninstall -y tensorflow
!pip3 install tensorflow==1.15.0rc2

!pip3 install h5py==2.10.0

Found existing installation: Keras 2.2.5
Uninstalling Keras-2.2.5:
  Successfully uninstalled Keras-2.2.5
  Using cached Keras-2.2.5-py2.py3-none-any.whl (336 kB)


## Import and Config

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import json

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

import Apex_points
from Apex_points.ray_tracing import localize_apices_engine as ray_tracing_engine
from Apex_points.skeleton import localize_apices_engine as skeleton_engine
from Apex_points.gradient import localize_apices_engine as gradient_engine

import traceback

Using TensorFlow backend.


In [ ]:
## UPDATE WT
class RadioConfig(Config):
    # Give the configuration a recognizable name
    NAME = "tooth_segmentation"
    
    NUM_CLASSES = 1 + 1 #NB : 2 because MT and JAD

    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    
    STEPS_PER_EPOCH = 250

    # New
    IMAGE_MAX_DIM = 256
    IMAGE_MIN_DIM = 256

config = RadioConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  256
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  256
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [256 256   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE             

## Inference production

In [ ]:
%cd ..

/content


In [ ]:
MODEL_DIR = ""
class InferenceConfig(RadioConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
model_path = "I3M.h5"
#model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead

Instructions for updating:
Use `tf.cast` instead.
Loading weights from  I3M.h5


In [ ]:
import numpy as np
from skimage.morphology import skeletonize
from skimage import exposure
from PIL import Image, ImageDraw


cnames = ['BG', 'MT']

In [ ]:
# Romain - add on : create dict
#
# Regis - edit: store results in the global variable as dictionnaries. dict_res is no longer a list but a dict, whose keys are the radio UNIQUE identifiers
# Advantages: no longer need to maintain two arrays side by side ('Radio_ID' and 'I3M') with the need to have same length and order

# Regis - 2021/07/05
# updated the apex_localize result dictionnary
dict_res = {}

def _combine_engine(engine_ids, image, mt_masks, debug=False, output_dir='.', image_name=''):
  import statistics
  from Apex_points.ray_tracing.image_merger import ImageMerger

  image_merger = ImageMerger()

  all_results = [engines[engine_id](image, mt_masks, debug=debug, output_dir=output_dir, image_name=image_name) for engine_id in engine_ids]
  combined_result = { 'I3M': -1, 'min_apex_opening': -1, 'max_apex_opening': -1, 'height': -1 }

  for i, engine_id in enumerate(engine_ids):
    output_image = all_results[i]['output_image'] if all_results[i]['output_image'] is not None else Image.new('RBG', (16, 16))
    image_merger.add_image(output_image, engine_id)

  apices_result = [apex_result for apex_result in all_results if apex_result is not None]
  for key in ['I3M', 'min_apex_opening', 'max_apex_opening', 'height']:
      combined_result[key] = statistics.mean([apex_result[key] for apex_result in apices_result if key in apex_result])
  result_image = image_merger.generate_image()
  combined_result['output_image'] = result_image
  if debug:
    result_image.save(os.path.join(output_dir, f'{image_name}-debug.png'))

  return combined_result

def _ray_tracing_plus_skeleton_engine(image, mt_masks, debug=False, output_dir='.', image_name=''):
  return _combine_engine(['Ray tracing', 'Skeleton'], image, mt_masks, debug, output_dir, image_name)

def _all_engine(image, mt_masks, debug=False, output_dir='.', image_name=''):
  return _combine_engine(['Ray tracing', 'Skeleton', 'Gradient'], image, mt_masks, debug, output_dir, image_name)

engines = { 'Ray tracing': ray_tracing_engine, 'Skeleton': skeleton_engine, 'Gradient': gradient_engine, 'Ray tracing + skeleton': _ray_tracing_plus_skeleton_engine, 'All': _all_engine }

def adaptHist(path_image): 
    img_to_pred_arr = np.array(Image.open(path_image).convert('RGB'))
    img_to_pred_adapthist = exposure.equalize_adapthist(img_to_pred_arr/np.max(img_to_pred_arr))
    img_to_pred_adapthist = Image.fromarray((img_to_pred_adapthist * 255).astype(np.uint8))
    return img_to_pred_adapthist

def make_predictions(image_paths):
  """
  Returns
  -------
    The result of the function `make_visualisations`
  """
  #images = [Image.open(image_path).convert('RGB') for image_path in image_paths]
  #names = [os.path.splitext(os.path.basename(filename))[0] for filename in image_paths]
  #np_images = [np.array(image) for image in images]
  
  if(HD=="Image already HD"):
    images = [Image.open(image_path).convert('RGB') for image_path in image_paths]
  else: 
    images = [adaptHist(image_path) for image_path in image_paths]

  names = [os.path.splitext(os.path.basename(filename))[0] for filename in image_paths]
  np_images = [np.array(image) for image in images]
  
  results = [model.detect([image], verbose=1) for image in np_images]
  return make_visualizations(results, images, names)

def make_visualizations(results, images, names):
   """
   Returns
   -------
   An array of the results returned by `make_prediction_image`
   """
   return [make_prediction_image(r[0], i, name) for r, i, name in zip(results, images, names)]

def make_prediction_image(r, img_to_pred, image_name, dict_res=dict_res):
  """
  Returns
  -------
    None in case of failure.
    Else: a tupple of :
    - the output image, as a PILLOW object.
    - the prediction engine result
    - the model result
    - an array of the mineralized tissue masks
  """
  # masks has a shape (height, width, number of detected classes)
  masks = r['masks']
  # masks is now an array with 'number of detected classes' elements
  # each element has a shape (height, width)
  masks_count = masks.shape[-1]
  masks = [mask.reshape((config.IMAGE_MAX_DIM, config.IMAGE_MAX_DIM)) for mask in np.split(masks, masks_count, axis=-1)]
  class_ids = r['class_ids']
  mt_masks = []
  for i, mask in enumerate(masks):
      current_class_name = cnames[class_ids[i]]
      if current_class_name == 'MT':
          mt_masks.append(mask)

  apices_result = Apex_points.localize_apices(img_to_pred, mt_masks, engines[endpoints_detection_method], debug, '.', image_name)
  if apices_result is not None:
    dict_res[image_name] = {}
    for key in ['I3M', 'min_apex_opening', 'max_apex_opening', 'height']:
      if key in apices_result:
        dict_res[image_name][key] = apices_result[key]
    return apices_result['output_image'], apices_result, r, mt_masks
  return None


In [ ]:
image_paths = image_filenames

In [ ]:
%cd /content
result_predictions = make_predictions(image_filenames)

result_images = []
for result in result_predictions:
  try: 
    result_images.append(result[0])
  except:
    result_images.append(None)
#result_images = [result[0] for result in result_predictions]

/content
Processing 1 images
image                    shape: (256, 256, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 256, 256, 3)      min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  256.00000  int64
anchors                  shape: (1, 16368, 4)         min:   -1.41976  max:    2.16878  float32
Processing 1 images
image                    shape: (256, 256, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 256, 256, 3)      min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  256.00000  int64
anchors                  shape: (1, 16368, 4)         min:   -1.41976  max:    2.16878  float32
Processing 1 images
image                    shape: (256, 256, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 256, 256, 3)      min: -123.70000  max:  

2021-07-26 14:50:23,309 - Apex points - ERROR - exception during treatment of 421_r: 2 MT masks are needed to infer
2021-07-26 14:50:23,316 - Apex points - ERROR - Traceback (most recent call last):
  File "/content/Mask_RCNN/Apex_points/utils.py", line 350, in localize_apices
    result = engine(image, mt_masks, debug, output_dir, image_name)
  File "/content/Mask_RCNN/Apex_points/gradient/gradient.py", line 202, in localize_apices_engine
    raise ValueError('2 MT masks are needed to infer')
ValueError: 2 MT masks are needed to infer

2021-07-26 14:50:30,226 - Apex points - ERROR - exception during treatment of 433_l: 2 MT masks are needed to infer
2021-07-26 14:50:30,228 - Apex points - ERROR - Traceback (most recent call last):
  File "/content/Mask_RCNN/Apex_points/utils.py", line 350, in localize_apices
    result = engine(image, mt_masks, debug, output_dir, image_name)
  File "/content/Mask_RCNN/Apex_points/gradient/gradient.py", line 202, in localize_apices_engine
    raise Val

In [ ]:
# save the results
![ -d Predictions ] || mkdir Predictions
for i, result_image in enumerate(result_images):
  if result_image is None:
    print(f'image at index {i} is None')
    continue
  try: 
    filename = image_filenames[i]
    image_name, image_extension = os.path.splitext(filename) # get the image name from the filename
    image_extension = image_extension[1:] # remove the leading dot

    output_path = os.path.join('Predictions', f'{image_name}-pred-{endpoints_detection_method}.{image_extension}')
    result_image.save(output_path, quality=100, subsampling=0)
    print(f'saved result at {output_path}')
  except Exception as exception:
    print(f'error in saving: {exception}')
    print(traceback.format_exc())

saved result at Predictions/414_l-pred-Gradient.jpg
saved result at Predictions/417_r-pred-Gradient.jpg
saved result at Predictions/418_r-pred-Gradient.jpg
saved result at Predictions/419_l-pred-Gradient.jpg
saved result at Predictions/420_r-pred-Gradient.jpg
image at index 5 is None
saved result at Predictions/427_l-pred-Gradient.jpg
saved result at Predictions/429_r-pred-Gradient.jpg
saved result at Predictions/432_l-pred-Gradient.jpg
image at index 9 is None
saved result at Predictions/439_r-pred-Gradient.jpg
saved result at Predictions/442_l-pred-Gradient.jpg
saved result at Predictions/444_l-pred-Gradient.jpg
saved result at Predictions/446_l-pred-Gradient.jpg
saved result at Predictions/454_r-pred-Gradient.jpg
saved result at Predictions/470_r-pred-Gradient.jpg
saved result at Predictions/472_l-pred-Gradient.jpg
saved result at Predictions/479_r-pred-Gradient.jpg
saved result at Predictions/484_l-pred-Gradient.jpg
image at index 19 is None
saved result at Predictions/486_r-pred-G

In [ ]:
if debug:
  _show_images()

## Comparison inference / manual (if applicable: see `run_mode`)

In [ ]:
from itertools import compress

In [ ]:
![ -d Manual_comparison ] || mkdir Manual_comparison

%matplotlib inline
# Regis - add on: compute area accuracy

import logging
import json

def _create_merged_mask(mt_masks):
  # create the composite image (logical or of all masks)
  mt_mask = np.zeros_like(mt_masks[0])
  for mask in mt_masks:
    mt_mask |= mask
  return mt_mask

def _shapes_to_masks(shapes, width, height):
  masks = []
  for shape in shapes:
    mask = Image.new('L', (width, height))
    points = [tuple(point) for point in shape['points']]
    ImageDraw.Draw(mask).polygon(points, fill=1)
    masks.append(np.array(mask))
  return masks

  masks = np.zeros((height, width))

def _json_file_to_mt_masks(json_filename):
    with open(json_filename) as json_file:
      annotations = json.load(json_file)
    shapes = annotations['shapes']
    mt_shapes = [shape for shape in shapes if shape['label'] == 'MT']
    mt_masks = _shapes_to_masks(mt_shapes, annotations['imageWidth'], annotations['imageHeight'])
    return mt_masks

def _json_file_to_mt_mask(json_filename):
    return _create_merged_mask(_json_file_to_mt_masks(json_filename))

def _number_pixels_in_mask(mask):
  """
  Parameters
  ----------
  mask: np.array
    A binary mask (or an int mask: any strict positive value will be considered as True).
  """
  return len(list(zip(*np.where(mask>0))))

def _compare_accuracy(ground_truth_mask, prediction_mask):
  """
  Compare pixels from the ground truth mask and the prediction one.

  Returns
  -------
  A tupple: (accuracy_image, stats)
  """
  union = ground_truth_mask | prediction_mask
  intersection = ground_truth_mask & prediction_mask

  tp_mask = intersection
  tn_mask = np.where(union>0,0,1)
  fp_mask = union & np.where(ground_truth_mask>0,0,1)
  fn_mask = union & np.where(prediction_mask>0,0,1)

  accuracy_image = Image.new('RGB', (union.shape[1], union.shape[0]))
  # TODO: refactoriser bibliothèque Apex_points pour ne plus appeler la fonction "privée" Apex_points.utils._apply_mask_to_image.
  Apex_points.utils._apply_mask_to_image(accuracy_image, tp_mask, (0, 255, 0), 1) # true positives: green
  Apex_points.utils._apply_mask_to_image(accuracy_image, tn_mask, (0, 0, 0), 1) # true negatives: black
  Apex_points.utils._apply_mask_to_image(accuracy_image, fn_mask, (255, 0, 0), 1) # false negatives (missing pixels): red
  Apex_points.utils._apply_mask_to_image(accuracy_image, fp_mask, (255, 128, 0), 1) # false positives: orange

  stat_masks = {'TP': tp_mask, 'TN': tn_mask, 'FP': fp_mask, 'FN': fn_mask}

  stats = {}
  for (key, mask) in stat_masks.items():
    stats[key] = _number_pixels_in_mask(mask)
  return (accuracy_image, stats)

def _compare_inference_against_manual():
  # load the manual segmentations
  # can zip image_filenames and result_predictions because they are assumed to have the same length (result_predictions is instantiated using the image_filenames array as the parameter of the function `make_predictions`)
  # Regis - 2021/07/05 plus qu'un seul appel à itertools.compress => on est certain de ne plus avoir de décalage
  for image_filename, result_prediction in compress(zip(image_filenames, result_predictions), [r is not None for r in result_predictions]):
    image_basename = image_filename.replace('.jpg', '').replace('.png', '')
    json_filename = f'{image_basename}.json'
    if not os.path.isfile(json_filename):
      logging.warning('ignoring missing manual segmentation file: {}'.format(json_filename))
      continue

    # mt masks is the third key of the prediction tupple
    prediction_mt_masks = Apex_points.utils._find_greatest_mt_masks(result_prediction[3])
    ground_truth_mt_masks = Apex_points.utils._find_greatest_mt_masks(_json_file_to_mt_masks(json_filename))
    accuracy_entries = [
        ('apical', ground_truth_mt_masks[0], prediction_mt_masks[0]),
        ('coronal', ground_truth_mt_masks[1], prediction_mt_masks[1]),
        ('global', ground_truth_mt_masks[0] | ground_truth_mt_masks[1], prediction_mt_masks[0] | prediction_mt_masks[1])
    ]

    for category, ground_truth_mt_mask, prediction_mt_mask in accuracy_entries:
      accuracy_image, stats = _compare_accuracy(ground_truth_mt_mask, prediction_mt_mask)
      accuracy_path = os.path.join('Manual_comparison', f'{image_basename}-{category}.png')
      accuracy_image.save(accuracy_path)

      for (key, value) in stats.items():
        dict_res[image_basename][f'{key}_{category}'] = value

    # blend image
    try:
      initial_image = Image.open(image_filename)
      global_accuracy_image = Image.open(os.path.join('Manual_comparison', f'{image_basename}-global.png'))
      blend_image = Image.blend(initial_image, global_accuracy_image, 0.4)
      blend_image.save(os.path.join('Manual_comparison', f'{image_basename}-blend.png'))
    except:
      pass

    # show the last accuracy image (because plt.imshow only display one image)
    if debug:
      plt.imshow(accuracy_image)

In [ ]:
if run_mode == 'Comparison inference / manual':
  _compare_inference_against_manual()

## I3M on manual segmentation (if applicable: see `run_mode`[)

In [ ]:
![ -d Manual_segmentation ] || mkdir Manual_segmentation

def _localize_apices_from_json_file_path(annotations, engine, debug=False, output_dir='.', image_name=''):
    # load the masks
    width, height = annotations['imageWidth'], annotations['imageHeight']
    mt_shapes = [shape for shape in annotations['shapes'] if shape['label'] == 'MT']
    mt_masks = [Apex_points.utils._labelme_shape_to_mask(shape, width, height) for shape in mt_shapes]
    image = Apex_points.utils._labelme_annotations_to_image(annotations)

    return Apex_points.utils.localize_apices(image, mt_masks, engine, debug, output_dir, image_name)
    
def _localize_apices_from_json_file_paths(json_file_paths, engine, debug=False, output_dir='.', engine_name='apex points'):
    import os
    import json

    # load all annotations
    result = {}
    all_annotations = {}
    for json_file_path in json_file_paths:
        if not os.path.exists(json_file_path):
            continue

        with open(json_file_path) as json_file:
            filename = os.path.splitext(os.path.basename(json_file_path))[0]
            try:
                all_annotations[filename] = json.load(json_file)
            except:
                continue
    if len(all_annotations) < 1:
        return []

    # perform analysis
    for filename, annotations in all_annotations.items():
        result[filename] = _localize_apices_from_json_file_path(annotations, engines[endpoints_detection_method], debug, output_dir, filename)
    return result
  
if run_mode == 'I3M on manual segmentation':
  manual_results = _localize_apices_from_json_file_paths(manual_segmentation_filenames, gradient_engine, debug=debug, output_dir='Manual_segmentation')
  #files.download('./Manual_segmentation/manual_results.csv')

## Store results as CSV

In [ ]:
# Romain - add on : write csv results
import csv

def _write_csv(keys, csv_output_path='./Predictions/results.csv', dict_res=dict_res):
  with open(csv_output_path, "w") as dict_res_file:
    writer = csv.writer(dict_res_file)
    writer.writerow(['Radio_ID'] + [CSV_HUMAN_KEYS[key] for key in keys])
    for radio_id in sorted(dict_res.keys()):
      values = [dict_res[radio_id][key] if key in dict_res[radio_id] else '' for key in keys]
      row = [radio_id]
      row.extend(values)
      writer.writerow(row)

def _csv_inference():
  _write_csv(['I3M', 'min_apex_opening', 'max_apex_opening', 'height'])

def _csv_comparison_inference_manual():
  _write_csv(['I3M', 'TP_apical', 'TN_apical', 'FP_apical', 'FN_apical', 'TP_coronal', 'TN_coronal', 'FP_coronal', 'FN_coronal', 'TP_global', 'TN_global', 'FP_global', 'FN_global', 'min_apex_opening', 'max_apex_opening', 'height'])

def _csv_manual_i3m():
  _write_csv(['I3M', 'min_apex_opening', 'max_apex_opening', 'height'], csv_output_path='./Manual_segmentation/manual_results.csv', dict_res=manual_results)

csv_functions = {
    'I3M on manual segmentation': _csv_manual_i3m,
    'Comparison inference / manual': _csv_comparison_inference_manual,
    'Inference': _csv_inference
}

CSV_HUMAN_KEYS = { 'I3M': 'I3M', 'TP_global': 'True positive', 'TN_global': 'True negative', 'FP_global': 'False positive', 'FN_global': 'False negative', 'TP_apical': 'True positive for apical mask', 'TN_apical': 'True negative for apical mask', 'FP_apical': 'False positive for apical mask', 'FN_apical': 'False negative for apical mask', 'TP_coronal': 'True positive for coronal mask', 'TN_coronal': 'True negative for coronal mask', 'FP_coronal': 'False positive for coronal mask', 'FN_coronal': 'False negative for coronal mask', 'min_apex_opening': 'Minimum apex opening (pixels)', 'max_apex_opening': 'Maximum apex opening (pixels)', 'height': 'Height (pixels)'}

csv_functions[run_mode]()

  ## Download final file

In [ ]:
if run_mode == 'I3M on manual segmentation':
  !zip -r manual_seg.zip Manual_segmentation/
  files.download('manual_seg.zip')
else : 
  !zip -r Predictions.zip Predictions/
  files.download('Predictions.zip')

updating: Predictions/ (stored 0%)
updating: Predictions/results.csv (deflated 48%)
updating: Predictions/417_r-pred-Gradient.jpg (deflated 2%)
  adding: Predictions/454_r-pred-Gradient.jpg (deflated 2%)
  adding: Predictions/418_r-pred-Gradient.jpg (deflated 3%)
  adding: Predictions/432_l-pred-Gradient.jpg (deflated 2%)
  adding: Predictions/420_r-pred-Gradient.jpg (deflated 2%)
  adding: Predictions/470_r-pred-Gradient.jpg (deflated 2%)
  adding: Predictions/429_r-pred-Gradient.jpg (deflated 2%)
  adding: Predictions/414_l-pred-Gradient.jpg (deflated 2%)
  adding: Predictions/446_l-pred-Gradient.jpg (deflated 2%)
  adding: Predictions/524_r-pred-Gradient.jpg (deflated 2%)
  adding: Predictions/419_l-pred-Gradient.jpg (deflated 2%)
  adding: Predictions/488_l-pred-Gradient.jpg (deflated 1%)
  adding: Predictions/510_l-pred-Gradient.jpg (deflated 2%)
  adding: Predictions/442_l-pred-Gradient.jpg (deflated 2%)
  adding: Predictions/472_l-pred-Gradient.jpg (deflated 2%)
  adding: Predic

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Last step: right click on Predictions.zip then download